In [ ]:
from collections import Counter
import heapq
from typing import Dict, List, Tuple

class HuffmanNode:
    def __init__(self, char: str = '', freq: int = 0):
        self.char = char
        self.freq = freq
        self.left = None
        self.right = None
        
    def __lt__(self, other):
        return self.freq < other.freq

class CompressionAnalyzer:
    def __init__(self, text: str):
        self.text = text
        self.freq_dict = Counter(text)
        self.huffman_codes = {}
        self.build_huffman_tree()
    
    def build_huffman_tree(self):
        # Create initial heap of nodes
        heap: List[HuffmanNode] = []
        for char, freq in self.freq_dict.items():
            heapq.heappush(heap, HuffmanNode(char, freq))
        
        # Build tree by combining nodes
        while len(heap) > 1:
            left = heapq.heappop(heap)
            right = heapq.heappop(heap)
            internal = HuffmanNode('', left.freq + right.freq)
            internal.left = left
            internal.right = right
            heapq.heappush(heap, internal)
            
        # Generate codes by traversing tree
        self._generate_codes(heap[0], "")
    
    def _generate_codes(self, node: HuffmanNode, code: str):
        if node.char:
            self.huffman_codes[node.char] = code
            return
        if node.left:
            self._generate_codes(node.left, code + "0")
        if node.right:
            self._generate_codes(node.right, code + "1")
    
    def analyze(self) -> Dict:
        # Calculate original bits
        original_bits = len(self.text) * 8
        
        # Calculate compressed bits
        compressed_bits = sum(len(self.huffman_codes[char]) * freq 
                            for char, freq in self.freq_dict.items())
        
        # Calculate compression ratio
        compression_ratio = ((original_bits - compressed_bits) / original_bits) * 100
        
        # Prepare character analysis
        char_analysis = []
        for char, freq in self.freq_dict.items():
            if char == ' ':
                char_display = 'space'
            elif char == '\n':
                char_display = '\\n'
            else:
                char_display = char
                
            char_analysis.append({
                'character': char_display,
                'occurrences': freq,
                'huffman_code': self.huffman_codes[char],
                'bits_used': len(self.huffman_codes[char]) * freq
            })
        
        # Sort by frequency descending
        char_analysis.sort(key=lambda x: x['occurrences'], reverse=True)
        
        return {
            'text_length': len(self.text),
            'unique_chars': len(self.freq_dict),
            'original_bits': original_bits,
            'compressed_bits': compressed_bits,
            'compression_ratio': round(compression_ratio, 2),
            'char_analysis': char_analysis
        }

def print_analysis(text: str):
    """Pretty print the compression analysis of a given text."""
    analyzer = CompressionAnalyzer(text)
    analysis = analyzer.analyze()
    
    print(f"\n=== Compression Analysis for: '{text}' ===\n")
    print(f"Text length: {analysis['text_length']} characters")
    print(f"Unique characters: {analysis['unique_chars']}")
    print(f"Original size: {analysis['original_bits']} bits")
    print(f"Compressed size: {analysis['compressed_bits']} bits")
    print(f"Compression ratio: {analysis['compression_ratio']}%")
    
    print("\nCharacter Analysis:")
    print("-" * 50)
    print(f"{'Char':^10} | {'Occurrences':^12} | {'Huffman Code':^12} | {'Bits':^6}")
    print("-" * 50)
    
    for char_data in analysis['char_analysis']:
        print(f"{char_data['character']:^10} | {char_data['occurrences']:^12} | "
              f"{char_data['huffman_code']:^12} | {char_data['bits_used']:^6}")
    print("-" * 50)

# Example usage
if __name__ == "__main__":
    sample_text = "zbbu buu zzbbeeebezb"
    print_analysis(sample_text)